# Coinbase Pro authenticated API queries

In [1]:
import numpy as np
import pandas as pd
from subprocess import Popen, PIPE, run
import json
from datetime import datetime

## Online resources
Coinbase Pro documentation:

https://docs.cloud.coinbase.com/exchange/docs/welcome

Coinbase Pro authorization and authentication:

https://docs.cloud.coinbase.com/exchange/docs/authorization-and-authentication

Coinbase Pro API reference guide:

https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getaccounts

In [2]:
# Coinbase Pro API endpoint:
endpoint = "https://api.exchange.coinbase.com"

In [3]:
# as a reminder, this is an unauthenticated query:
# Get all Coinbase Pro trading pairs:
cmd = [
    "curl",
    "%s/products"%endpoint,
    "--header",
    "Accept: application/json",
    "--header",
    "Content-Type: application/json",
    ]
p = Popen(cmd,stdout=PIPE,stderr=PIPE)
stdout,stderr = p.communicate()

# parse stdout json data:
products_query = json.loads(stdout)

# display to user:
products_query[0]

{'id': 'BTRST-USD',
 'base_currency': 'BTRST',
 'quote_currency': 'USD',
 'base_min_size': '0.07',
 'base_max_size': '250000',
 'quote_increment': '0.001',
 'base_increment': '0.01',
 'display_name': 'BTRST/USD',
 'min_market_funds': '1',
 'max_market_funds': '100000',
 'margin_enabled': False,
 'fx_stablecoin': False,
 'max_slippage_percentage': '0.03000000',
 'post_only': False,
 'limit_only': False,
 'cancel_only': False,
 'trading_disabled': False,
 'status': 'online',
 'status_message': '',
 'auction_mode': False}

In [5]:
# now let's add authenticated requests:
# First,we need three pieces of authentication information:
# 1. API key
# 2. API secret
# 3. API passphrase 
#
# more info at:
# https://docs.cloud.coinbase.com/exchange/docs/authorization-and-authentication
# 
# I'll load them in using a .secret file
# (.secret files are ignored by this git repository)
with open("../bin/crypto-api-example-key.secret","r") as of:
    API_KEY = of.readline().rstrip()
    API_SECRET = of.readline().rstrip()
    API_PASSPHRASE = of.readline().rstrip()

In [6]:
# let's read in the example portfolio's current accounts:
# endpoint: https://api.exchange.coinbase.com/accounts
# 
# to do this we need to use the API key, secret, and passphrase
# to generate four headers:
# 1. CB-ACCESS-KEY
# 2. CB-ACCESS-SIGN
# 3. CB-ACCESS-TIMESTAMP
# 4. CB-ACCESS-PASSPHRASE
#
# The key and passphrase -- headers one and four -- are simply 
# the key and passphrase we generated. The sign and timestamp 
# however take some explaining. 
#
# first, the CB-ACCESS-SIGN explanation copied from Coinbase
# website: 
# The CB-ACCESS-SIGN header is generated by creating a sha256 
# HMAC using the base64-decoded secret key on the prehash string 
# timestamp + method + requestPath + body (where + represents 
# string concatenation) and base64-encode the output. The 
# timestamp value is the same as the CB-ACCESS-TIMESTAMP header.
# The body is the request body string or omitted if there is no 
# request body (typically for GET requests).
#
# API authentification modules:
import time
import hmac
import hashlib
import base64

# signature generation function:
def timestamp_and_signature(
    method,
    api_secret,
    request_path,
    body,
    ):
    hmac_key = base64.b64decode(api_secret)
    timestamp = str(time.time())
    message = timestamp + method + request_path + body
    message = message.encode('ascii')
    signature = hmac.new(
        hmac_key,
        message,
        hashlib.sha256,
        )
    signature_b64 = base64.b64encode(
        signature.digest()
        ).decode('utf-8')
    return timestamp, signature_b64

# create the timestamp and the signature:
accounts_url = "https://api.exchange.coinbase.com/accounts"
timestamp, sign = timestamp_and_signature(
    "GET",
    API_SECRET,
    "/accounts",
    "", #no body message necessary
    )

# define required commands that don't require 
# authentication:
unauth_cmd = [
    "curl",
    accounts_url,
    "--header",
    "Accept: application/json",
    "--header",
    "Content-Type: application/json",
    ]

# define the required authentication commands:
auth_cmd = [
    "--header",
    "CB-ACCESS-KEY: %s"%API_KEY,
    "--header",
    "CB-ACCESS-SIGN: %s"%sign,
    "--header",
    "CB-ACCESS-TIMESTAMP: %s"%timestamp,
    "--header",
    "CB-ACCESS-PASSPHRASE: %s"%API_PASSPHRASE,
    ]

# submit request:
accounts_cmd = unauth_cmd + auth_cmd
p = Popen(accounts_cmd,stdout=PIPE,stderr=PIPE)
stdout,stderr = p.communicate()
accounts_output = json.loads(stdout)

In [7]:
accounts_output[0]

{'id': '01834e69-2cfb-42d6-b33a-b7e4bbdd4c97',
 'currency': '1INCH',
 'balance': '0.0000000000000000',
 'hold': '0.0000000000000000',
 'available': '0',
 'profile_id': '476602b3-b9df-4e6a-85e5-f92af99afeb5',
 'trading_enabled': True}

In [8]:
# let's get the accounts with non-zero balances:
# function to do so: 
def extract_balances(output):
    nonzero_accounts = []
    for account in output:
        balance = float(account["balance"])
        if balance > 0.0:
            nonzero_accounts.append(account)
    return nonzero_accounts

# actual extraction:
nonzero_accounts = extract_balances(accounts_output)
nonzero_accounts

[{'id': '9f14f06a-8a1d-4254-be65-c01b3b7fbccc',
  'currency': 'ATOM',
  'balance': '0.0000660000000000',
  'hold': '0.0000000000000000',
  'available': '0.000066',
  'profile_id': '476602b3-b9df-4e6a-85e5-f92af99afeb5',
  'trading_enabled': True},
 {'id': 'af5da678-ad3c-436e-b077-91bf38e6a88f',
  'currency': 'USDC',
  'balance': '1.0082260000000000',
  'hold': '0.0000000000000000',
  'available': '1.008226',
  'profile_id': '476602b3-b9df-4e6a-85e5-f92af99afeb5',
  'trading_enabled': True},
 {'id': '3babb006-d122-41ca-bf27-cfb783b10382',
  'currency': 'XLM',
  'balance': '2.0000000000000000',
  'hold': '0.0000000000000000',
  'available': '2',
  'profile_id': '476602b3-b9df-4e6a-85e5-f92af99afeb5',
  'trading_enabled': True}]

In [12]:
# convert into pandas' dataframe for easier 
# analysis. Make sure to update the column
# data types as necessary: 
df = pd.DataFrame(nonzero_accounts)
for col in ["balance","hold","available"]:
    df.loc[:,col] = df[col].astype(float)

# index by currency and keep only the balance,
# hold, and available columns:
df = df.set_index("currency")
df = df.loc[:,["balance","hold","available"]]
    
# show:
df

,balance,hold,available
currency,,,
ATOM,0.000066,0.0,0.000066
USDC,1.008226,0.0,1.008226
XLM,2.000000,0.0,2.000000
